In [3]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# --- 1. Carregamento e tratamento dos dados ---
df = pd.read_csv('dados/diabetes.csv')
df.fillna(df.median(), inplace=True)

# Colunas usadas para predição (exemplo)
feature_cols = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

X = df[feature_cols]
y = df['Outcome']

# --- 2. Treinar modelo preditivo (Random Forest) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Opcional: print relatório para você analisar (não será mostrado no Dash)
print(classification_report(y_test, model.predict(X_test)))

# --- 3. Inicializar app Dash ---
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True  # <-- ESSA LINHA
app.title = "Dashboard Completo Diabetes"

# --- 4. Layout com abas ---
app.layout = html.Div([
    html.H1("Dashboard Completo Diabetes"),
    dcc.Tabs(id="tabs", value='tab-exploration', children=[
        dcc.Tab(label='Exploração de Dados', value='tab-exploration'),
        dcc.Tab(label='Visualização Avançada', value='tab-visualization'),
        dcc.Tab(label='Predição', value='tab-prediction'),
    ]),
    html.Div(id='tabs-content')
])

# --- 5. Conteúdo das abas ---
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-exploration':
        return html.Div([
    html.H3('Exploração de Dados'),
    
    html.Label("Selecione a faixa etária:"),
    dcc.RangeSlider(
        id='age-slider',
        min=df['Age'].min(),
        max=df['Age'].max(),
        step=1,
        value=[df['Age'].min(), df['Age'].max()],
        marks={i: str(i) for i in range(int(df['Age'].min()), int(df['Age'].max())+1, 10)},
        tooltip={"placement": "bottom", "always_visible": True},
    ),
    
    html.Br(),
    html.H4("Gráfico 1: Glicose por Idade (Scatter)"),
    dcc.Graph(id='graph-glucose-age'),

    html.Br(),
    html.H4("Gráfico 2: Distribuição da Idade por Diagnóstico"),
    dcc.Graph(id='graph-age-distribution'),

    html.Br(),
    html.H4("Gráfico 3: Boxplot de IMC por Diagnóstico"),
    dcc.Graph(id='graph-bmi-boxplot'),

    html.Br(),
    html.H4("Gráfico 4: Correlação das Variáveis (Heatmap)"),
    dcc.Graph(id='graph-correlation-heatmap'),

], style={'padding': 20})

    
    elif tab == 'tab-visualization':
        return html.Div([
            html.H3('Visualização Avançada'),
            dcc.Dropdown(
                id='feature-dropdown',
                options=[{'label': col, 'value': col} for col in feature_cols],
                value='Glucose',
                clearable=False,
                style={'width': '50%'}
            ),
            dcc.Graph(id='feature-distribution'),
        ], style={'padding': 20})
    
    elif tab == 'tab-prediction':
        return html.Div([
            html.H3('Predição de Diabetes'),
            html.P("Insira os valores para prever se a pessoa tem diabetes:"),
            
            html.Div([
                html.Label('Número de gestações:'),
                dcc.Input(id='input-pregnancies', type='number', min=0, step=1, value=0),
            ]),
            html.Div([
                html.Label('Glicose:'),
                dcc.Input(id='input-glucose', type='number', min=0, step=1, value=120),
            ]),
            html.Div([
                html.Label('Pressão Arterial:'),
                dcc.Input(id='input-bloodpressure', type='number', min=0, step=1, value=70),
            ]),
            html.Div([
                html.Label('Espessura da Pele:'),
                dcc.Input(id='input-skinthickness', type='number', min=0, step=1, value=20),
            ]),
            html.Div([
                html.Label('Insulina:'),
                dcc.Input(id='input-insulin', type='number', min=0, step=1, value=79),
            ]),
            html.Div([
                html.Label('IMC:'),
                dcc.Input(id='input-bmi', type='number', min=0, step=0.1, value=32.0),
            ]),
            html.Div([
                html.Label('Função de Pedigree para Diabetes:'),
                dcc.Input(id='input-dpf', type='number', min=0, step=0.01, value=0.5),
            ]),
            html.Div([
                html.Label('Idade:'),
                dcc.Input(id='input-age', type='number', min=0, step=1, value=30),
            ]),
            html.Button('Prever', id='btn-predict', n_clicks=0, style={'marginTop': 10}),
            html.H4(id='prediction-output', style={'marginTop': 20, 'color': 'blue'})
        ], style={'padding': 20, 'maxWidth': 400})
    
    else:
        return html.Div()

# --- 6. Callbacks para aba Exploração ---
@app.callback(
    Output('graph-glucose-age', 'figure'),
    Output('graph-age-distribution', 'figure'),
    Output('graph-bmi-boxplot', 'figure'),
    Output('graph-correlation-heatmap', 'figure'),
    Input('age-slider', 'value')
)
def update_exploration_graphs(age_range):
    low, high = age_range
    filtered_df = df[(df['Age'] >= low) & (df['Age'] <= high)]

    # Gráfico 1: Glicose por Idade
    fig1 = px.scatter(
        filtered_df, x='Age', y='Glucose', color='Outcome',
        labels={'Outcome': 'Diabetes'},
        title='Glicose por Idade'
    )

    # Gráfico 2: Histograma de Idade
    fig2 = px.histogram(
        filtered_df, x='Age', color='Outcome', barmode='overlay',
        nbins=30, title='Distribuição da Idade por Diagnóstico'
    )
    fig2.update_traces(opacity=0.7)

    # Gráfico 3: Boxplot de IMC
    fig3 = px.box(
        filtered_df, x='Outcome', y='BMI', points="all",
        labels={'Outcome': 'Diabetes', 'BMI': 'IMC'},
        title='Boxplot de IMC por Diagnóstico'
    )

    # Gráfico 4: Heatmap de Correlação
    corr_matrix = filtered_df[feature_cols + ['Outcome']].corr()
    fig4 = px.imshow(
        corr_matrix,
        text_auto=True,
        aspect="auto",
        color_continuous_scale='RdBu_r',
        title='Matriz de Correlação'
    )

    return fig1, fig2, fig3, fig4


# --- 7. Callbacks para aba Visualização Avançada ---
@app.callback(
    Output('feature-distribution', 'figure'),
    Input('feature-dropdown', 'value')
)
def update_visualization(selected_feature):
    fig = px.histogram(
        df,
        x=selected_feature,
        color='Outcome',
        barmode='overlay',
        nbins=30,
        labels={'Outcome': 'Diabetes'},
        title=f'Distribuição da variável {selected_feature} por diagnóstico'
    )
    fig.update_traces(opacity=0.75)
    return fig

# --- 8. Callback para aba Predição ---
@app.callback(
    Output('prediction-output', 'children'),
    Input('btn-predict', 'n_clicks'),
    State('input-pregnancies', 'value'),
    State('input-glucose', 'value'),
    State('input-bloodpressure', 'value'),
    State('input-skinthickness', 'value'),
    State('input-insulin', 'value'),
    State('input-bmi', 'value'),
    State('input-dpf', 'value'),
    State('input-age', 'value'),
)
def make_prediction(n_clicks, preg, gluc, bp, skin, insul, bmi, dpf, age):
    if n_clicks == 0:
        return ""
    
    # Criar dataframe para predição
    data = pd.DataFrame({
        'Pregnancies': [preg],
        'Glucose': [gluc],
        'BloodPressure': [bp],
        'SkinThickness': [skin],
        'Insulin': [insul],
        'BMI': [bmi],
        'DiabetesPedigreeFunction': [dpf],
        'Age': [age]
    })
    
    # Prever
    pred = model.predict(data)[0]
    prob = model.predict_proba(data)[0][pred]
    
    if pred == 1:
        return f"⚠️ O modelo indica que há risco de diabetes (Probabilidade: {prob:.2f})"
    else:
        return f"✅ O modelo indica que NÃO há risco de diabetes (Probabilidade: {prob:.2f})"

# --- 9. Executar o app ---
if __name__ == '__main__':
    app.run_server(debug=True)


              precision    recall  f1-score   support

           0       0.79      0.78      0.78        99
           1       0.61      0.62      0.61        55

    accuracy                           0.72       154
   macro avg       0.70      0.70      0.70       154
weighted avg       0.72      0.72      0.72       154

